In [45]:
#!py -m pip install pyodbc pandas db-dtypes
#!py -m pip install google-cloud-bigquery google-auth google-auth-oauthlib pandas_gbq

In [46]:
import pyodbc
import pandas as pd
import pandas_gbq
import json
from google.cloud import bigquery
from google.api_core.exceptions import NotFound

In [47]:
# Open and read the JSON file
with open('config/mom_db_config.json', 'r') as file:
    mom_db_config = json.load(file)

In [48]:
MOM_CONN_STR = (
    f"{mom_db_config['driver']}"
    f"{mom_db_config['server']}"
    f"{mom_db_config['database']}"
    f"{mom_db_config['user']}"
    f"{mom_db_config['password']}"
)
PROJECT_ID = "sturdy-tome-468208-k2"
DATASET_ID = "dev"
TABLE_IDS = {
    "orders" : "orders"
}

In [49]:
# Create connection
mom_conn = pyodbc.connect(MOM_CONN_STR)

In [50]:
# Initialize client (uses default credentials)
bq_client = bigquery.Client(project=PROJECT_ID)

In [51]:
# Get the latest order id
query = f"""
    SELECT MAX(order_id) AS max_id FROM `{DATASET_ID}.{TABLE_IDS['orders']}` 
"""
max_id = None
try :
    # Run query and return results as a DataFrame
    df = bq_client.query(query).to_dataframe()
    if len(df) > 0 :
        max_id = df.iloc[0]['max_id']
except NotFound:
    print(f"Table not found: {TABLE_IDS['orders']}")

c:\Users\danny.sloan\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [52]:
max_id

np.float64(1588631.0)

In [53]:
last_order_time  = "2025-08-06 12:27:05.000"
last_order_num = 1588550

params = []
parms_str = ""
if max_id is not None:
    params = [max_id]
    parms_str = " WHERE orderno > ? "

query = f"""
    SELECT
        orderno AS order_id,
        custnum AS customer_id,
        entrytime AS entry_time,
        ord_total AS order_total
        FROM CMS 
        {parms_str}
        ORDER BY order_id ASC
"""

df = pd.read_sql(query, mom_conn, params = params)

C:\Users\danny.sloan\AppData\Local\Temp\ipykernel_7624\3755343262.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, mom_conn, params = params)


In [ ]:
# Convert ids to integers
df['order_id'] = df['order_id'].astype(int)
df['customer_id'] = df['customer_id'].astype(int)

In [54]:
df.shape

(2, 4)

In [55]:
df.tail(10)

,order_id,customer_id,entry_time,order_total
0,1588632.0,935857.0,2025-08-06 15:41:14,35.0
1,1588633.0,509752.0,2025-08-06 15:43:32,105.0


In [56]:
# We need to fix the null entrytimes, just backwards fill from the previous seen entrytime
df['entry_time'] = pd.to_datetime(df['entry_time'], errors='coerce')  # ensure datetime type
df['entry_time'] = df['entry_time'].fillna(method='ffill')  # forward-fill

C:\Users\danny.sloan\AppData\Local\Temp\ipykernel_7624\1582709845.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['entry_time'] = df['entry_time'].fillna(method='ffill')  # forward-fill


In [57]:
df.tail(10)

,order_id,customer_id,entry_time,order_total
0,1588632.0,935857.0,2025-08-06 15:41:14,35.0
1,1588633.0,509752.0,2025-08-06 15:43:32,105.0


In [58]:
mom_conn.close()

In [59]:
#!gcloud auth application-default login

In [ ]:
# Configure load job — let schema be auto-detected
job_config = bigquery.LoadJobConfig(
    autodetect = True,              # This makes schema generation automatic
    write_disposition = "WRITE_APPEND"  # Or WRITE_TRUNCATE / WRITE_EMPTY
)

# Load DataFrame into BigQuery
job = bq_client.load_table_from_dataframe(
    df,
    f"{DATASET_ID}.{TABLE_IDS['orders']}",
    job_config=job_config
)
job.result()  # Wait for the load to complete

print("Loaded {} rows into {}".format(df.shape[0], TABLE_IDS['orders']))

Loaded 2 rows into orders


: 

In [ ]:
query = f"""
    SELECT * FROM `{DATASET_ID}.{TABLE_IDS['orders']}`
"""

# Run query and return results as a DataFrame
df = bq_client.query(query).to_dataframe()

c:\Users\danny.sloan\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\cloud\bigquery\table.py:1965: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [ ]:
df.shape

(1510066, 4)

In [ ]:
df.head(10)

,order_id,customer_id,entry_time,order_total
0,3.0,42852.0,2004-03-31 13:35:46,21.98
1,4.0,42850.0,2004-03-31 15:43:24,11.75
2,11.0,570323.0,2004-04-01 14:26:23,17.98
3,12.0,42873.0,2004-04-01 14:50:33,41.97
4,13.0,42872.0,2004-04-01 15:37:32,26.97
5,15.0,42875.0,2004-04-01 16:10:36,26.97
6,21.0,311247.0,2004-04-01 17:38:31,26.97
7,31.0,446458.0,2004-04-02 13:31:58,26.97
8,38.0,42904.0,2004-04-05 12:09:29,31.98
9,40.0,42900.0,2004-04-05 12:24:55,9.99
